In [1]:
from pyspark.sql.functions import explode,to_timestamp,regexp_replace


StatementMeta(, 746b01db-4065-4ce9-bdca-24b648c60698, 3, Finished, Available, Finished)

In [2]:
df = spark.read.option("multiline", "true").json("Files/Directory/google-latest-news.json")
display(df)

StatementMeta(, 746b01db-4065-4ce9-bdca-24b648c60698, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, cf2c8cc3-30cd-4a29-8e21-ff5d788e1cd9)

In [3]:

df_transform = df.select("news_results").withColumn("news",explode(df.news_results)).select("news")
display(df_transform)

StatementMeta(, 746b01db-4065-4ce9-bdca-24b648c60698, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 68288903-4c45-490d-b75a-fbc81af724dc)

In [4]:
df_explode = df_transform.withColumn("position",df_transform.news.position)\
                            .withColumn("title",df_transform.news.title)\
                            .withColumn("date",to_timestamp(regexp_replace(df_transform.news.iso_date,"[A-Z]",""),"yyyy-MM-ddHH:mm:ss"))\
                            .withColumn("source",df_transform["news.source.name"])\
                            .withColumn("link",df_transform.news.link)\
                            .withColumn("icon",df_transform.news.thumbnail_small)\
                        .drop("news")
display(df_explode)

StatementMeta(, 746b01db-4065-4ce9-bdca-24b648c60698, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 0737780c-28bb-4774-a923-4d0d8fc49d17)

In [5]:
from pyspark.sql.utils import AnalysisException
try:
    table_name = "tbl_latest_news"

    df_explode.write.format("delta").saveAsTable(table_name)
except AnalysisException:
    print("Table already exists")
    df_explode.createOrReplaceTempView("vw_latest_news")
    spark.sql(f"""MERGE INTO {table_name} tgt USING vw_latest_news src
                ON src.link = tgt.link
                WHEN MATCHED AND
                src.title <> tgt.title OR
                src.date <> tgt.date OR
                src.source <> tgt.source OR
                src.icon <> tgt.icon
                THEN UPDATE SET *
                WHEN NOT MATCHED
                THEN INSERT *
            """)

StatementMeta(, 746b01db-4065-4ce9-bdca-24b648c60698, 7, Finished, Available, Finished)

Table already exists


In [6]:
%%sql 
select * from dbo.tbl_latest_news

StatementMeta(, 746b01db-4065-4ce9-bdca-24b648c60698, 8, Finished, Available, Finished)

<Spark SQL result set with 100 rows and 6 fields>